<a href="https://colab.research.google.com/github/lilianabs/benchmarking-dl-frameworks/blob/main/Flux_mlp_fashionmnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots Flux MLDatasets"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.7.1 on the current Colab Runtime...
2022-04-05 22:40:19 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.7/julia-1.7.1-linux-x86_64.tar.gz [123374573/123374573] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...
Installing Julia package Flux...
Installing Julia package MLDatasets...
Installing Julia package CUDA...


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, haswell)
Environment:
  JULIA_NUM_THREADS = 2


In [2]:
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

  478.899 ms (2 allocations: 32.00 MiB)


In [3]:
if ENV["COLAB_GPU"] == "1"
    using CUDA

    run(`nvidia-smi`)

    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
else
    println("No GPU found.")
end

Tue Apr  5 22:50:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
using Flux
using Flux.Data: DataLoader
using Flux: onehotbatch, onecold, @epochs
using Flux.Losses: logitcrossentropy
using MLDatasets
using Plots
using CUDA

In [5]:
ENV["DATADEPS_ALWAYS_ACCEPT"] = "true"

"true"

In [6]:
# load full training set
train_x, train_y = FashionMNIST.traindata(Float32)

# load full test set
test_x,  test_y  = FashionMNIST.testdata(Float32)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [9, 2, 1, 1, 6, 1, 4, 6, 5, 7  …  5, 6, 8, 9, 1, 9, 1, 8, 1, 5])

In [7]:
size(train_x)

(28, 28, 60000)

In [8]:
size(train_y)

(60000,)

In [9]:
unique(train_y)

10-element Vector{Int64}:
 9
 0
 3
 2
 7
 5
 1
 6
 4
 8

In [10]:
train_x = Flux.flatten(train_x)
test_x = Flux.flatten(test_x)

784×10000 Matrix{Float32}:
 0.0  0.0         0.0         …  0.0       0.0       0.0       0.0
 0.0  0.0         0.0            0.0       0.0       0.0       0.0
 0.0  0.0         0.0            0.0       0.0       0.0       0.0
 0.0  0.0         0.0            0.0       0.0       0.0       0.0
 0.0  0.0         0.0            0.0       0.0       0.0       0.0
 0.0  0.0         0.0         …  0.0       0.0       0.0       0.0
 0.0  0.0         0.0            0.0       0.0       0.0       0.0
 0.0  0.0         0.0            0.0       0.0       0.0       0.0
 0.0  0.0         0.00392157     0.0       0.0       0.0       0.0
 0.0  0.0         0.0            0.121569  0.0       0.0       0.0
 0.0  0.0509804   0.262745    …  0.203922  0.0       0.0       0.0
 0.0  0.262745    0.694118       0.384314  0.0       0.643137  0.0
 0.0  0.0         0.505882       0.368627  0.0       0.537255  0.0
 ⋮                            ⋱                                
 0.0  0.00784314  0.266667       0.537

In [11]:
# One-hot encode the labels
train_y, test_y = onehotbatch(train_y, 0:9), onehotbatch(test_y, 0:9)

(Bool[0 1 … 1 0; 0 0 … 0 0; … ; 0 0 … 0 0; 1 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 1 0; … ; 0 0 … 0 0; 1 0 … 0 0])

In [12]:
# Create DataLoaders (mini-batch iterators)
train_data_loader = DataLoader((train_x, train_y), batchsize=64, shuffle=true)
test_data_loader = DataLoader((test_x, test_y), batchsize=64)

DataLoader{Tuple{Matrix{Float32}, Flux.OneHotArray{UInt32, 10, 1, 2, Vector{UInt32}}}, Random._GLOBAL_RNG}((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Bool[0 0 … 0 0; 0 0 … 1 0; … ; 0 0 … 0 0; 1 0 … 0 0]), 64, 10000, true, 10000, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 10000], false, Random._GLOBAL_RNG())

In [28]:
CUDA.allowscalar(false)
device = gpu

gpu (generic function with 1 method)

In [30]:
# Construct model
img_size = (28,28,1)
num_classes = 10

model = Chain( Dense(prod(img_size), 512, relu),
               Dense(512, 512, relu),
               Dense(512, num_classes)) 

ps = Flux.params(model) # model's trainable parameters

# model |> device

Params([Float32[-0.054709226 -0.051337138 … 0.026578406 -0.0023194444; 0.010374862 0.014308265 … 0.0592415 -0.06364672; … ; -0.05236719 0.032355674 … -0.014872518 -0.021380195; 0.019930512 -0.044395424 … 0.017219808 -0.0358684], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.052696817 -0.04587356 … 0.06883099 -0.037374232; -0.010613331 -0.017085772 … 0.017432123 -0.05198341; … ; -0.049667403 -0.017132776 … -0.008835577 -0.03189518; -0.029269582 0.053982846 … 0.03230454 -0.034464203], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.01585538 -0.07165226 … -0.09283537 0.019271884; -0.036619294 -0.05282642 … -0.06798605 -0.095837064; … ; 0.05426361 0.03317631 … -0.001011979 -0.010703018; 0.018936804 0.0585916 … -0.1001359 -0.09097651], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])

In [14]:
function loss(data_loader, model)
    total_loss = 0.0f0
    num_elements = 0
    for (x, y) in data_loader
        ŷ = model(x)
        total_loss += logitcrossentropy(ŷ, y, agg=sum)
        num_elements +=  size(x)[end]
    end
    return total_loss / num_elements
end


function accuracy(data_loader, model)
    accuracy = 0
    num_elements = 0
    for (x, y) in data_loader
        ŷ = model(x)
        accuracy += sum(onecold(ŷ) .== onecold(y))
        num_elements += size(x)[end]
    end   
    
    return accuracy / num_elements
end

accuracy (generic function with 1 method)

In [15]:
η = 1e-3 

opt = Descent(η)

Descent(0.001)

In [19]:
function train()
  epochs = 5

  train_loss_results = []
  test_loss_results = []
  train_accuracy_results = []
  test_accuracy_results = []

  for epoch in 1:epochs
      for (x, y) in train_data_loader
          gs = gradient(() -> logitcrossentropy(model(x), y), ps) # Compute gradient
          Flux.Optimise.update!(opt, ps, gs) # Update parameters
      end
          
      # Compute accuracy and loss for all of the train and test data
      train_loss = loss(train_data_loader, model)
      train_acc = accuracy(train_data_loader, model)
      test_loss = loss(test_data_loader, model)
      test_acc = accuracy(test_data_loader, model)
      println("Epoch=$epoch")
      println("  train_loss = $train_loss, train_accuracy = $train_acc")
      println("  test_loss = $test_loss, test_accuracy = $test_acc")
      push!(train_loss_results, train_loss)
      push!(test_loss_results, test_loss)
      push!(train_accuracy_results, train_acc)
      push!(test_accuracy_results, test_acc)
  end
end

train (generic function with 1 method)

In [20]:
@time train()

Epoch=1
  train_loss = 0.6431834, train_accuracy = 0.7963666666666667
  test_loss = 0.6614916, test_accuracy = 0.7831
Epoch=2
  train_loss = 0.61620647, train_accuracy = 0.8061
  test_loss = 0.63583106, test_accuracy = 0.7905
Epoch=3
  train_loss = 0.59294116, train_accuracy = 0.8119166666666666
  test_loss = 0.6130389, test_accuracy = 0.7982
Epoch=4
  train_loss = 0.57469404, train_accuracy = 0.8159
  test_loss = 0.5951256, test_accuracy = 0.8018
Epoch=5
  train_loss = 0.5588081, train_accuracy = 0.8203333333333334
  test_loss = 0.5801001, test_accuracy = 0.8079
 72.188560 seconds (2.32 M allocations: 26.898 GiB, 4.88% gc time, 0.29% compilation time)
